In [14]:
import numpy as np
import cv2 as cv
import glob
from constants import *
import os
from time import sleep
world_footage_path = os.path.join(recordings_folder, "camera_calibrations","world_footage")
depth_footage_path = os.path.join(recordings_folder, "camera_calibrations","depth_footage")

## record depth camera

In [2]:
"""
import pyrealsense2 as rs
#record frames into depth footage path as png's

depth_footage_path =depth_footage_path + "_2"
print(depth_footage_path)
rs_config = rs.config()
rs_config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 15)

pipeline = rs.pipeline()
pipeline.start(rs_config)


for i in range(20):
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    color_image = np.asanyarray(color_frame.get_data())
    cv.imwrite(os.path.join(depth_footage_path, str(i) + ".png"), color_image)

#only keep 20% of the frames
depth_frames = glob.glob(os.path.join(depth_footage_path, "*.png"))
depth_frames.sort(key=os.path.getmtime)
for i in range(len(depth_frames)):
    if i % 5 != 0:
        os.remove(depth_frames[i])
"""

C:/Users/mathi/Documents/GitHub/gaze_data_acquisition/mathis/recordings/camera_calibrations\depth_footage_2


: 

In [4]:
"""
#extract frames from world footage path as png's
world_cam_path = os.path.join(world_footage_path,"world_vid.mp4")
video = cv.VideoCapture(world_cam_path)
fps = video. get(cv. CAP_PROP_FPS)
print(fps)
i = 0
while True:
    ret, frame = video.read()
    if not ret:
        break
    if i %12 == 0:
        cv.imwrite(os.path.join(world_footage_path, str(i) + ".png"), frame)
    i += 1
"""


30.026383606797022


## world camera

In [5]:
#calibrate the world camera with checkerboard images with cv2
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((8*11,3), np.float32)
objp[:,:2] = np.mgrid[0:8,0:11].T.reshape(-1,2)*60                        #60mm between each square
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob(os.path.join(world_footage_path,'*.png'))
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (8,11), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (8,11), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

In [7]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
#print all the calibration parameters
print("ret: ", ret)
print("mtx: ", mtx)
print("dist: ", dist)
print("rvecs: ", rvecs)
print("tvecs: ", tvecs)

ret:  0.325259142496391
mtx:  [[757.52356581   0.         548.84273484]
 [  0.         758.05514165 539.54825932]
 [  0.           0.           1.        ]]
dist:  [[-0.340153    0.17212689  0.00064248  0.00134531  0.0156342 ]]
rvecs:  (array([[ 0.43195008],
       [ 0.17326403],
       [-1.48404255]]), array([[ 0.2214916 ],
       [ 0.2609044 ],
       [-1.62428014]]), array([[ 0.54833807],
       [ 0.05735206],
       [-1.42323475]]), array([[ 0.2430816 ],
       [ 0.22957633],
       [-1.58695265]]), array([[ 0.28423145],
       [ 0.19303374],
       [-1.55451967]]), array([[ 0.31020647],
       [ 0.0966573 ],
       [-1.56040064]]), array([[ 0.32934928],
       [ 0.00693279],
       [-1.55438458]]), array([[ 0.41450797],
       [-0.0194504 ],
       [-1.5273012 ]]), array([[ 0.3724626 ],
       [-0.04822276],
       [-1.50510404]]), array([[ 0.2474025 ],
       [-0.17500359],
       [-1.51881049]]), array([[ 0.18870717],
       [-0.21265244],
       [-1.53968523]]), array([[ 0.2238

### calibrate distorsion

In [9]:
img = cv.imread(os.path.join(world_footage_path, '108.png'))
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [10]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
calibration_path = os.path.join(recordings_folder, "camera_calibrations","calibration_world.png")
cv.imwrite(calibration_path, dst)

True

In [11]:
# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
calibration_path = os.path.join(recordings_folder, "camera_calibrations","calibration_world_2.png")
cv.imwrite(calibration_path, dst)

True

## Depth camera

In [18]:
#calibrate the world camera with checkerboard images with cv2
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((8*11,3), np.float32)
objp[:,:2] = np.mgrid[0:8,0:11].T.reshape(-1,2)*60                        #60mm between each square
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob(os.path.join(depth_footage_path,'*.png'))
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (8,11), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (8,11), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

In [19]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
#print all the calibration parameters
print("ret: ", ret)
print("mtx: ", mtx)
print("dist: ", dist)
print("rvecs: ", rvecs)
print("tvecs: ", tvecs)

ret:  0.18172094223444762
mtx:  [[610.88622433   0.         314.66214168]
 [  0.         612.29341612 250.33631586]
 [  0.           0.           1.        ]]
dist:  [[-1.29888314e-02  1.29486325e+00  1.66978628e-03  3.40160858e-04
  -5.25260849e+00]]
rvecs:  (array([[ 0.28042749],
       [-0.58138443],
       [-1.49569071]]), array([[ 0.42078158],
       [-0.62360987],
       [-1.45670158]]), array([[ 0.29273702],
       [-0.47114341],
       [-1.46590522]]), array([[ 0.1678887 ],
       [-0.3518077 ],
       [-1.44488639]]), array([[-0.05513107],
       [-0.22693008],
       [-1.41750544]]), array([[-0.22237335],
       [-0.03123571],
       [-1.36859448]]), array([[-0.36282971],
       [ 0.0661867 ],
       [-1.31594206]]), array([[ 0.25470126],
       [-0.55181713],
       [-1.48288636]]), array([[-0.41983458],
       [ 0.11307207],
       [-1.31577625]]), array([[-0.6205629 ],
       [ 0.27501126],
       [-1.21467501]]), array([[-0.65551056],
       [ 0.38064894],
       [-1.1409

In [ ]:
img = cv.imread(os.path.join(depth_footage_path, '100.png'))
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [ ]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
calibration_path = os.path.join(recordings_folder, "camera_calibrations","calibration_depth.png")
cv.imwrite(calibration_path, dst)

In [ ]:
# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
calibration_path = os.path.join(recordings_folder, "camera_calibrations","calibration_depth_2.png")
cv.imwrite(calibration_path, dst)